The goal of the homework is to study, based on actual production data, the notion of "peak oil" introduced in lecture 1

#### Administrativia
Groups composition to be provided no later than Wednesday, Sep. 25, 8 PM (by email)\
GROUPS SHALL DIFFER IN MEMBERSHIP FROM HOMEWORK #1\
Homework to be turned in Thursday, Oct. 3, 10 PM (deposit .ipynb file in Moodle / General Informations, only 1 submission per group)\
Please indicate here the names of the group's members:

#### Data set
One CSV file, Oil_production.csv taken from https://www.energyinst.org/statistical-review/resources-and-data-downloads \
Description: crude oil production in million tonnes by country/region (col.A) and year\
NOTE: this is a good example of using "real world" imperfect data sets which have not been prepared for machine processing\
IMPORTANT: Your program shall take as input this csv file, without manual cleaning/preparation

#### Data preparation
Study the data set and identify what cleanup/preparation steps are needed \
Remove all "Total" lines and remove USSR (it appears after the 1991 breakup as its member countries)

In [83]:
import pandas as pd 
df = pd.read_csv('Oil_production.csv', skiprows=1,encoding='latin1', delimiter=';')
print(f'original csv file: {df.head()}')

# drop first two rows
df = df.drop(df.index[:2])
print(f'first two rows dropped: {df.head()}')

# Keep all rows"Total" lines, lines withut country names,and remove USSR
df_filtered = df[
    (~df.iloc[:,0].str.contains('Total', case=False, na=False)) &
    (~df.iloc[:,0].str.contains('USSR', case=False, na=False)) &
    (df.iloc[:,0].notna())
]

df = df.reset_index(drop=True)
df_filtered.to_csv('Oil_production_cleaned.csv', index=False, float_format='%.1f', sep=';', decimal='.')

original csv file:   Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
0    Country       1965       1966       1967       1968       1969   
1        NaN        NaN        NaN        NaN        NaN        NaN   
2     Canada       43,9       48,2       52,7       57,1       62,2   
3     Mexico       18,1       18,5       20,5       21,9         23   
4         US      427,7      454,5      484,2      502,9      511,4   

  Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 53 Unnamed: 54  \
0       1970       1971       1972       1973  ...        2017        2018   
1        NaN        NaN        NaN        NaN  ...         NaN         NaN   
2       70,1       75,2       86,7      100,3  ...       236,6       257,7   
3       24,2       24,1       25,1       25,9  ...       109,5       102,3   
4      533,5      525,9      527,9      514,7  ...       574,2       669,6   

  Unnamed: 55 Unnamed: 56 Unnamed: 57 Unnamed: 58 Unnamed: 59  \
0        2019       

#### Data processing
Write a function which takes as input :
1. time series (floats)
2.   a threshold (float)

and returns true if the last two data points of the time series are more than threshold % lower than the maximum value in the series

In [84]:
# series is a number indicating a row in csv
def has_peaked(series, threshold):
    df = pd.read_csv('Oil_production_cleaned.csv', delimiter=';', header=0)
    row = df.iloc[series,1:-3] # choose all columns except the country column in given rows
    #df_list = df.tolist()
    numeric_row = row.apply(lambda x: float(str(x).replace(',', '.')) if isinstance(x, str) else x)

    max_value = numeric_row.max()
    sec_last = numeric_row[-5] 
    last = numeric_row[-4] # two last columns in every chosen row
    if ((max_value - last)/ max_value > threshold and (max_value - sec_last)/ max_value > threshold):
        return True # production peaked
    else:
        return False # production not peaked
# returns bool

# Check
has_peaked(3, 0.5)


/var/folders/96/qj751xx557q28_4ctxx82jlw0000gn/T/ipykernel_16032/461408190.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sec_last = numeric_row[-5]
/var/folders/96/qj751xx557q28_4ctxx82jlw0000gn/T/ipykernel_16032/461408190.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last = numeric_row[-4] # two last columns in every chosen row


False

##### First graph
Plot on Y axis the percentage (0 to 1) of all oil-producing countries whose recent production (last 2 years) is more than threshold % lower than their peak as a function of threshold, with threshold ranging from 0 to 100 (X axis)\
How many countries have not peaked yet?

#### Second graph
Plot a stacked vertical bar graph with each country on the X axis (sorted by decreasing value of the country's maximum production). Each bar displays both the 2023  production and the maximum production (in a lighter color) of the country\
Bonus points if the bar's color encodes the oil-producing region the country belongs to.